<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Webscraping" data-toc-modified-id="Webscraping-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Webscraping</a></span></li><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data cleaning</a></span></li><li><span><a href="#Export-dataframe-from-Python-into-MySQL" data-toc-modified-id="Export-dataframe-from-Python-into-MySQL-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Export dataframe from Python into MySQL</a></span></li></ul></div>

# Webscraping

In [1]:
from requests.exceptions import SSLError, Timeout, TooManyRedirects, RequestException
from contextlib import closing
from IPython.display import Markdown, display
import warnings
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
import re

from time import sleep #To delay the time between each loop in order to avoid webscraping alert by the website

In [215]:
warnings.filterwarnings('ignore')
timeout_glob=0.05
verify_glob=True
allow_glob=False

# Request headers helps avoid the blocking by the website that consider our resquest is from a bot
headers = """accept: text/javascript, text/html, application/xml, text/xml, */*
accept-encoding: gzip, deflate, br
accept-language: vi-VN,vi;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5
cookie: frontend=8hn6paqnu8bk406101ro0l3g9t; t2s-p=182428c9-043a-46da-cd5e-2b7cc7729851; frontend_cid=BRccL8qPmWfWqpqa; t2s-userId=SXoKSdoT; _gcl_au=1.1.386306217.1599579941; _ga=GA1.2.1796094192.1599579942; _cs_c=1; gaWidgetPrx=GA1.2.1796094192.1599579942; _fbp=fb.1.1599579941943.1513295207; Pastease.passive.activated.Mb6i5rT6EDi9OOt=0; Pastease.passive.chance.Mb6i5rT6EDi9OOt=chance0.3; __gads=ID=08237db4e7038bd9:T=1599579942:S=ALNI_MbLOnZPQvVlxyTBuSg9AxgyYEcKDA; OptanonAlertBoxClosed=2020-09-08T15:57:03.866Z; _gid=GA1.2.706547475.1599741549; m_ses=20200910143909; gaWidgetPrx_gid=GA1.2.1056287777.1599741550; fstrz_vary=novaluenovalue; _uetsid=3ac3aa6c5b32498234c07330110c97c2; _uetvid=f73125c81a13ec851a4a0856fceb2819; _cs_id=abc280bd-9056-a559-95a3-b508aa554206.1599579941.14.1599834047.1599832574.1.1633743941686.Lax.0; _cs_s=2.1; t2s-rank=rank3; m_cnt=42; OptanonConsent=isIABGlobal=false&datestamp=Fri+Sep+11+2020+16%3A20%3A48+GMT%2B0200+(heure+d%E2%80%99%C3%A9t%C3%A9+d%E2%80%99Europe+centrale)&version=5.13.0&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&hosts=&geolocation=%3B&AwaitingReconsent=false; ABTasty=uid=0gpxspxr4kpzwhqz&fst=1599579941786&pst=1599741550179&cst=1599815443029&ns=4&pvt=90&pvis=90&th=484325.609150.90.26.4.1.1599579941827.1599834065164.1_610831.756242.32.15.2.1.1599579941834.1599596131986.1_613659.760620.90.26.4.1.1599579941837.1599834065171.1_622734.0.58.26.2.1.1599741550308.1599834065176.1; ABTastySession=mrasn=&lp=https://www.go-sport.com/fitness-musculation/musculation/?p=1&is_ajax=1&sen=163
referer: https://www.go-sport.com/nouveautes/
sec-fetch-dest: empty
sec-fetch-mode: cors
sec-fetch-site: same-origin
user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36
x-newrelic-id: VQ4AU1RXDBABVFZTBwIHUFU=
x-prototype-version: 1.7
x-requested-with: XMLHttpRequest"""

headers=dict([i.strip().split(': ') for i in headers.split('\n')])

# 1. Functions to check errors while pulling data from website

def printmd(string):
    display(Markdown('**'+string+'**'))
    
#Function to check Code response (200), type file (json), content of the file is not none   
def is_good_response(x):
    """
    Returns True if the response seems to be an HTML, False otherwise
    x is response
    """
    content_type=x.headers['Content-Type'].lower()

    return ((x.status_code==200 and content_type is not None and content_type.find('json')>1), x.status_code)
#Function to print type error
def log_error(e):
    print(e)

#Function to print the checking message according to the response code from the website 
def status_handler(status):
    if status<300:
        print('Alles gut!')
    elif status>=400 and status<500:
        print('request has failed due to you mistake bro. Check the link, headers, server and whether you have an access to this page.')
    else:
        print('Yoooo, I have no clue what has happened. But you are stuck, since you didnt receive any output from server')
    return None

# 2. Function to read json file gotten from Urls and retrieve the part in html file that contains the information about products, then store data in the "list_product" to import into Dataframe 
def lst_data(result,list_product):
    soup = BeautifulSoup(result.json()['page']).select('li')
    for n in soup:    
        adaptive_scraping(url_updated, list_product)
        list_product.append(dict(re.findall(r'(\b[a-zA-Z_-]+)=?\"(.*?)\"', str(n))))
        sleep(3)

# 3. Function to get the url, then combine all functions above in one
def adaptive_scraping(url,list_product, timeout=timeout_glob, verify=verify_glob):
    global timeout_glob
    global verify_glob
    global allow_glob
    import requests as r
    try:
        with closing(r.get(url, headers = headers, stream=True, timeout=timeout_glob, verify=verify_glob, allow_redirects=allow_glob)) as resp:
            test1=is_good_response(resp)
            lst_data(resp,list_product)
            print(resp)
            if test1[0]:
                return resp.content
            else:
                return status_handler(test1[1])
    except Timeout as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('We can increase response waiting time. Do you want to continue?')
        ans=input('\n[Yes|No]\t')
        if ans.lower()=='yes':
            timeout_glob*=2
            return adaptive_scraping(url,list_product)
        else:
            return None
    except SSLError as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('We can skip verification test. Do you want to continue?')
        ans=input('\n[Yes|No]\t')
        if ans.lower()=='yes':
            verify_glob=False
            return adaptive_scraping(url,list_product)
        else:
            return None
    except TooManyRedirects as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('We can increase the number of allowed redirects. Do you want to continue?')
        ans=input('\n[Yes|No]\t')
        if ans.lower()=='yes':
            allow_glob=True
            return adaptive_scraping(url,list_product)
        else:
            return None
    except RequestException as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('Unfortunately, we have no clue what to do.  Please try again later. ')
        return None

In [3]:
#Loop applied to the global function 'url_updated' to retrieve all data from all pages of the website
url_sample ='https://www.go-sport.com/nouveautes/?p=XXX&src=mm&is_ajax=1'

list_product = []

for n in range(1,107):
    url_updated = url_sample.replace('XXX',str(n))
    adaptive_scraping(url_updated, list_product)
    
df = pd.DataFrame(list_product)

df.head(1)

,class,data-best-sku,data-brand,data-category,data-color,data-crossed-price,data-in-stock,data-name,data-position,data-price,...,data-picture,data-srcset,type,data-placeholder,data-src,onerror,id,style,trail,short
0,c-thumbnail-product__content__price__new,000000014735250101,NIKE,Football / Clubs / Ligue 1 Française / PSG,Bleu,,True,PSG DOMICILE 2020-2021,0,89.99,...,1,https://medias.go-sport.com/media/resized/340x...,image/webp,/skin/frontend/gosport/default/images/catalog/...,https://medias.go-sport.com/media/resized/340x...,this.onerror=null;initOnErrorMedias(this);,product-price-5054431,NaN,NaN,NaN


In [4]:
df.shape

(2544, 28)

In [5]:
# To store the raw data in excel file.
df.to_excel('D:\Documents\Ironhack\Projet 2\go_sport_nouveautés.xlsx', index = False)

# Data cleaning

**Dataframe structure check**
* 27 columns and 2544 rows (same as the number of the products seen on the website)


**Column check** 

* Out of 27 columns, we note that there are 17 unnecessary columns or duplicate columns to drop

* Rename column names

* Change of the order of several columns

* Reset the index and add a name to the index column

**Value check**

* Missing value check: there are 3 columns containing missing values
            Category                   176: Missing data on the website. Solution: drop or let it be.
            Color                      561: It is normal that there are products whose colors do not need to be precised (nutrition, bodybuilding equipment,...). Thus, leave NaN. 
            Original Price            2484: Only discounted products (not including the club discounted product) have values in this column. Thus, leave NaN

* Flatten the mix categories column by split them in different subcategory levels (check word spelling)

* Based on the information in the column 'Category', we create a new column called'Age Group' containing 4 values: Infant, Children/Teens and Adult and Not precised if NaN.

* Based on the information in the column 'Category', we create a new column called 'Gender' containing 3 values: M and F and Not precised if NaN.
    
* Create a new column called "Club discounted product": add 'Yes' if the product has the club member discount, otherwise add  'No'

* Deal with the column 'Category'



In [32]:
import pandas as pd

In [ ]:
def data_cleaning(df):
    
    # Column check: from the column analysis , there are below list of columns to drop
    drop_col = ['class','data-best-sku','data-position','data-picture','data-srcset','type','data-placeholder','data-route-name','data-src','href','onerror','title','style','trail','short']
    df.drop(drop_col, axis = 1, inplace = True)
    df.drop(columns = ['alt','id'], inplace = True)
    
    # Column check: Rename column names
    df.rename(columns = {'data-brand':'Brand', 'data-category':'Category', 'data-color':'Color', 'data-crossed-price':'Original Price',
       'data-in-stock':'Buy Online', 'data-name':'Name Product', 'data-price':'Current Price', 'data-price-excl-tax':'Current Price Excl VAT',
       'data-shop-name':'Vendor', 'data-sku':'Id Product', 'data_is_promo':'Discounted Product'}, inplace = True)
    
    # Reorder the position of columns of the dataframe
    df = df[['Id Product','Name Product','Brand', 'Category', 'Color', 'Original Price', 'Buy Online', 
         'Current Price', 'Current Price Excl VAT', 'Vendor', 'Discounted Product']]
    
    # Reset the index so that it starts from 1
    ix = pd.Index(range(1,df.shape[0]+1), name ='Number')
    df.set_index(ix, inplace = True)
    
    #We create a new column called "Club discounted product": add 'Yes' if the product has the club member discount, otherwise add  'No'
    df.loc[(df['Original Price'].isna() == True) & (df['Discounted Product'] == True), 'Club Discounted Product'] = 'Yes'
    df.loc[~((df['Original Price'].isna() == True) & (df['Discounted Product'] == True)), 'Club Discounted Product'] = 'No'
    
    # Since the 'Category' columns contain information about 'Age group'
    # Create a new column:'Age Group' containing 4 values: Infant, Children/Teens and Adult and Not precised if NaN.
    df.loc[df['Category'].str.contains('hommes?|femmes?', na = False, case = False),'Age Group'] = 'Adult'
    df.loc[df['Category'].str.contains('gar[cç]ons?|filles?|enfants?', na = False, case = False),'Age Group'] = 'Children/Teens'
    df.loc[df['Category'].str.contains('b[ée]b[ée]', na = False, case = False),'Age Group'] = 'Infant'
    age_group_na = {'Age Group':'Not precised'}
    df.fillna(value = age_group_na, inplace = True)
    
    # Since the 'Category' columns contain information about 'Gender'
    # Create a new column:'Gender' containing 3 values: M and F and Not precised if NaN.
    df.loc[df['Category'].str.contains('hommes?|gar[cç]ons?', na = False, case = False),'Gender'] = 'M'
    df.loc[df['Category'].str.contains('femmes?|filles?', na = False, case = False),'Gender'] = 'F'
    gender_na = {'Gender':'Not precised'}
    df.fillna(value = gender_na, inplace = True)

In [2]:
# Read the raw data file

df = pd.read_excel(r'D:\Documents\Ironhack\Projet 2\go_sport_nouveautés.xlsx')

In [3]:
df.columns

Index(['class', 'data-best-sku', 'data-brand', 'data-category', 'data-color',
       'data-crossed-price', 'data-in-stock', 'data-name', 'data-position',
       'data-price', 'data-price-excl-tax', 'data-route-name',
       'data-shop-name', 'data-sku', 'data_is_promo', 'href', 'title', 'alt',
       'data-picture', 'data-srcset', 'type', 'data-placeholder', 'data-src',
       'onerror', 'id', 'style', 'trail', 'short'],
      dtype='object')

In [4]:
# Column check: to check what columns contain unnecessary values
list_col = df.columns

for n in list_col:
    unique_val = df[n].unique()
    nb_unique_val = df[n].nunique()
    print(f'The column "{n}" has {nb_unique_val} unique values: \n {unique_val} \n')

The column "class" has 4 unique values: 
 ['c-thumbnail-product__content__price__new'
 'c-thumbnail-product__content__price__club--label'
 'c-thumbnail-product__content__price__initial'
 'c-thumbnail-product__content__price__old'] 

The column "data-best-sku" has 1934 unique values: 
 [1.47352501e+10 1.48079701e+10 1.14705601e+10 ... 1.47995201e+10
 3.60768252e+12 4.01791205e+12] 

The column "data-brand" has 141 unique values: 
 ['NIKE' 'EASTPAK' 'PUMA' 'WANABEE' 'ELLESSE' 'ADIDAS' 'ASICS' 'REEBOK'
 'UP2GLIDE' 'THE NORTH FACE' 'ROSSIGNOL' 'GARMIN' 'CONVERSE' 'TIMBERLAND'
 'UHLSPORT' 'LEVIS' 'ATHLITECH' 'BROOKS' 'TEDDY SMITH' 'SOFTWR' 'TACCHINI'
 'SKECHERS' 'QUIKSILVER' 'BOXEUR DES RUES' 'FREEGUN' 'PLASTIMO' 'NEW ERA'
 'ONLY' 'THUASNE SPORT' 'IPANEMA' 'BIO TECH USA' 'LE COQ SPORTIF'
 'BABOLAT' 'JACK &amp; JONES' 'SALOMON' 'COLUMBIA' 'UNDER ARMOUR' 'FILA'
 'STIGA' '1ER PRIX' 'SCRAPPER' 'OPTIMUM NUTRITION' 'ISOSTAR' 'TRIGANO'
 'USN NUTRITION' 'OVERSTIMS' 'HEAD' 'HOKA ONE ONE' 'CAIRN' 'AR

In [5]:
# Column check: from the analysis above, there are below list of columns to drop

drop_col = ['class','data-best-sku','data-position','data-picture','data-srcset','type','data-placeholder','data-route-name','data-src','href','onerror','title','style','trail','short']

df.drop(drop_col, axis = 1, inplace = True)

In [6]:
df.head()


,data-brand,data-category,data-color,data-crossed-price,data-in-stock,data-name,data-price,data-price-excl-tax,data-shop-name,data-sku,data_is_promo,alt,id
0,NIKE,Football / Clubs / Ligue 1 Française / PSG,Bleu,NaN,True,PSG DOMICILE 2020-2021,89.99,89.99,GSP,000000000001473525,False,PSG DOMICILE 2020-2021,product-price-5054431
1,NIKE,Football / Clubs / Ligue 1 Française / PSG,Blanc,NaN,True,PSG MAILLOT EXTERIEUR 20,89.99,89.99,GSP,000000000001480797,False,PSG MAILLOT EXTERIEUR 20,product-price-5562756
2,EASTPAK,Accessoires / Sacs &amp; Besaces / Sacs À Dos,Noir,NaN,True,PADDED NOIR,49.99,49.99,GSP,000000000001147056,True,PADDED NOIR,NaN
3,EASTPAK,Accessoires / Sacs &amp; Besaces / Sacs À Dos,Gris,NaN,True,PADDED GRIS,49.99,49.99,GSP,000000000001112232,True,PADDED GRIS,NaN
4,PUMA,Football / Clubs / Ligue 1 Française,Bleu ciel,NaN,True,OM MAILLOT EXTERIEUR 20,89.99,89.99,GSP,000000000001475281,False,OM MAILLOT EXTERIEUR 20,product-price-5213989


In [7]:
# Check if column contains values that have only white space character.

df[df['alt'].str.contains(r'^\s*$') == True]

df[df['data-name'].str.contains(r'^\s*$') == True]


# As we can see, there is no value in these two column that contains only white space character

,data-brand,data-category,data-color,data-crossed-price,data-in-stock,data-name,data-price,data-price-excl-tax,data-shop-name,data-sku,data_is_promo,alt,id


In [8]:
# Check if there are any duplicated values between the two columns. If they are duplicates of each other, we need to drop one of them
count_dup = 0
for n in range(df.shape[0]):
    if df.loc[n,'data-name'] == df.loc[n,'alt']:
        count_dup+=1
count_dup

#there are 2544 duplicated values which are the same number of rows of table -> drop the column 'alt'

2544

In [9]:
#Drop supplementary unnecessary columns

df.drop(columns = ['alt','id'], inplace = True)

In [10]:
# Column check: Rename column names
df.rename(columns = {'data-brand':'Brand', 'data-category':'Category', 'data-color':'Color', 'data-crossed-price':'Original Price',
       'data-in-stock':'Buy Online', 'data-name':'Name Product', 'data-price':'Current Price', 'data-price-excl-tax':'Current Price Excl VAT',
       'data-shop-name':'Vendor', 'data-sku':'Id Product', 'data_is_promo':'Discounted Product'}, inplace = True)

In [11]:
df.sample(5)

,Brand,Category,Color,Original Price,Buy Online,Name Product,Current Price,Current Price Excl VAT,Vendor,Id Product,Discounted Product
1602,FREETIME,Sports / Randonnée / Tentes / Tente Camping,NaN,NaN,True,MOUNTAIN 1 DLX - tentes légére 1.7 Kg - tente ...,119.70,99.750000,Freetime-boutique,3660323302143,False
2140,LOEFFLER,Running / Trail / Vetements / Short - Cuissard...,Multicolore,89.99,False,Loeffler 3/4 Basic,77.98,64.983333,Tradeinn,2448|136415855,True
1346,TEDDY SMITH,Chaussures / Homme Loisirs / Sneakers / Chauss...,Marron,NaN,True,DRAMEN,59.99,59.990000,GSP,000000000001480003,False
953,ODLO,Sports / Sports d'Hiver / Tenues de Ski / Homm...,Gris,NaN,True,ACTIV WARM,39.99,39.990000,GSP,000000000001381148,False
81,NIKE,Chaussures / Enfant Loisirs / Bébé / Chaussure...,Blc/ros,NaN,True,COURT BOROUGH LOW 2 SE VLC,34.99,34.990000,GSP,000000000001479498,False


In [12]:
df.columns

Index(['Brand', 'Category', 'Color', 'Original Price', 'Buy Online',
       'Name Product', 'Current Price', 'Current Price Excl VAT', 'Vendor',
       'Id Product', 'Discounted Product'],
      dtype='object')

In [13]:
# Reorder the position of columns of the dataframe
df = df[['Id Product','Name Product','Brand', 'Category', 'Color', 'Original Price', 'Buy Online', 
         'Current Price', 'Current Price Excl VAT', 'Vendor', 'Discounted Product']]

In [14]:
df.head(5)

,Id Product,Name Product,Brand,Category,Color,Original Price,Buy Online,Current Price,Current Price Excl VAT,Vendor,Discounted Product
0,000000000001473525,PSG DOMICILE 2020-2021,NIKE,Football / Clubs / Ligue 1 Française / PSG,Bleu,NaN,True,89.99,89.99,GSP,False
1,000000000001480797,PSG MAILLOT EXTERIEUR 20,NIKE,Football / Clubs / Ligue 1 Française / PSG,Blanc,NaN,True,89.99,89.99,GSP,False
2,000000000001147056,PADDED NOIR,EASTPAK,Accessoires / Sacs &amp; Besaces / Sacs À Dos,Noir,NaN,True,49.99,49.99,GSP,True
3,000000000001112232,PADDED GRIS,EASTPAK,Accessoires / Sacs &amp; Besaces / Sacs À Dos,Gris,NaN,True,49.99,49.99,GSP,True
4,000000000001475281,OM MAILLOT EXTERIEUR 20,PUMA,Football / Clubs / Ligue 1 Française,Bleu ciel,NaN,True,89.99,89.99,GSP,False


In [15]:
# Reset the index so that it starts from 1

ix = pd.Index(range(1,df.shape[0]+1), name ='Number')

df.set_index(ix, inplace = True)

In [16]:
df.head()

,Id Product,Name Product,Brand,Category,Color,Original Price,Buy Online,Current Price,Current Price Excl VAT,Vendor,Discounted Product
Number,,,,,,,,,,,
1,000000000001473525,PSG DOMICILE 2020-2021,NIKE,Football / Clubs / Ligue 1 Française / PSG,Bleu,NaN,True,89.99,89.99,GSP,False
2,000000000001480797,PSG MAILLOT EXTERIEUR 20,NIKE,Football / Clubs / Ligue 1 Française / PSG,Blanc,NaN,True,89.99,89.99,GSP,False
3,000000000001147056,PADDED NOIR,EASTPAK,Accessoires / Sacs &amp; Besaces / Sacs À Dos,Noir,NaN,True,49.99,49.99,GSP,True
4,000000000001112232,PADDED GRIS,EASTPAK,Accessoires / Sacs &amp; Besaces / Sacs À Dos,Gris,NaN,True,49.99,49.99,GSP,True
5,000000000001475281,OM MAILLOT EXTERIEUR 20,PUMA,Football / Clubs / Ligue 1 Française,Bleu ciel,NaN,True,89.99,89.99,GSP,False


In [17]:
df.isna().sum()

Id Product                   0
Name Product                 0
Brand                        0
Category                   176
Color                      561
Original Price            2484
Buy Online                   0
Current Price                0
Current Price Excl VAT       0
Vendor                       0
Discounted Product           0
dtype: int64

In [18]:
df[df['Original Price'].isna() == False]

,Id Product,Name Product,Brand,Category,Color,Original Price,Buy Online,Current Price,Current Price Excl VAT,Vendor,Discounted Product
Number,,,,,,,,,,,
10,000000000001453700,JOGGING GARDA CUT AND SEW,ELLESSE,Vêtements / Homme Loisir / Pantalons &amp; jog...,Gris,44.99,True,17.99,17.990000,GSP,True
11,000000000001457918,KARL 2,ELLESSE,Chaussures / Homme Loisirs / Sneakers / Chauss...,Marine,69.99,True,34.99,34.990000,GSP,True
349,000000000001435430,FIT 1PC CB Y,ADIDAS,Sports / Natation - Piscine / Maillot de Bain ...,Bleu marine,24.99,True,12.49,12.490000,GSP,True
508,000000000001428570,GUARRY STY NOIR,ATHLITECH,Vêtements / Homme Sport / Autres Sports / Nata...,Noir,12.99,True,6.49,6.490000,GSP,True
837,000000000001455369,BE BOLD GRAPHIC 7/8,PUMA,Vêtements / Femme Sport / Fitness - Yoga / Bas...,Noir,49.99,True,34.99,29.158333,Shopping Factory,True
899,000000000001473811,TOTTENHAM DRIL TOP 20,NIKE,NaN,Rose,64.99,True,45.99,38.325000,Shopping Factory,True
1087,000000000001478999,M NK CHLLGR SHORT 7IN 2IN1,NIKE,Vêtements / Homme Sport / Running - Trail / Ba...,Noir/gri,34.99,True,24.99,20.825000,Shopping Factory,True
1173,000000000001483657,PIKE LAKE,COLUMBIA,Vêtements / Femme Sport / Randonnée - Montagne...,Noir,199.99,True,163.99,136.658333,Shopping Factory,True
1246,3660323301825,RAIDLITE 1 DLX - Petite tente - Tente légère 1...,FREETIME,Sports / Randonnée / Tentes / Tente Camping,NaN,139.90,True,119.10,99.250000,Freetime-boutique,True


In [19]:
# Check the coherence between the values of the column "Original Price " and the ones in "Discounted product":
# Theorically, the products whose original prices are NaN, are non discounted products

count_error = 0

for n in range(1,df.shape[0]):
    if df['Original Price'].isna().loc[n] == True and df.loc[n,'Discounted Product'] == True:
        count_error+=1
    elif df['Original Price'].isna().loc[n] == False and df.loc[n,'Discounted Product'] == False:
        count_error+=1
        
print(count_error)

#As we can see, there are 135 products that have an incoherence between the values of "Original Price" and "Discounted Product"

135


In [20]:
#We extract here the list of products that should not have been mentioned "discounted"
df[(df['Original Price'].isna() == True) & (df['Discounted Product'] == True)]

#By analyzing the webpages of go-sport, we note that there are another type of discount which is called "Club discount" when you have a club member card. 
#The products in this category of discount have the current price appeared on the website with a discounted price.
#We create a new column called "Club discounted product"

,Id Product,Name Product,Brand,Category,Color,Original Price,Buy Online,Current Price,Current Price Excl VAT,Vendor,Discounted Product
Number,,,,,,,,,,,
3,000000000001147056,PADDED NOIR,EASTPAK,Accessoires / Sacs &amp; Besaces / Sacs À Dos,Noir,NaN,True,49.99,49.99,GSP,True
4,000000000001112232,PADDED GRIS,EASTPAK,Accessoires / Sacs &amp; Besaces / Sacs À Dos,Gris,NaN,True,49.99,49.99,GSP,True
8,000000000001339484,PADDED BLEU JEAN,EASTPAK,Accessoires / Sacs &amp; Besaces / Sacs À Dos,Bleu,NaN,True,49.99,49.99,GSP,True
12,000000000001458227,KARL 2,ELLESSE,Chaussures / Homme Loisirs / Sneakers / Chauss...,Noir,NaN,True,69.99,69.99,GSP,True
13,000000000001453698,Sweat - ARNO CUT AND SEW HOODY,ELLESSE,Marques / Nos Sélections / Ellesse,Noir,NaN,True,54.99,54.99,GSP,True
...,...,...,...,...,...,...,...,...,...,...,...
2088,000000000001479014,M NK CHLLGR SHORT FF PR,NIKE,Vêtements / Homme Sport / Running - Trail / Ba...,Noir/gri,NaN,False,34.99,34.99,GSP,True
2225,000000000001480858,W NK AIR SHORT,NIKE,Vêtements / Femme Sport / Running - Trail / Ba...,Noir/gris,NaN,False,29.99,29.99,GSP,True
2269,000000000001479479,REVOLUTION 5 VLC,NIKE,Chaussures / Enfant Loisirs / Multisport / Cha...,Marine,NaN,False,37.99,37.99,GSP,True


In [21]:
#We create a new column called "Club discounted product": add 'Yes' if the product has the club member discount, otherwise add  'No'
df.loc[(df['Original Price'].isna() == True) & (df['Discounted Product'] == True), 'Club Discounted Product'] = 'Yes'

In [22]:
#We create a new column called "Club discounted product": add 'Yes' if the product has the club member discount, otherwise add  'False'
df.loc[~((df['Original Price'].isna() == True) & (df['Discounted Product'] == True)), 'Club Discounted Product'] = 'No'

In [23]:
# Since the 'Category' columns contain information about 'Age group'
# Create a new column:'Age Group' containing 4 values: Infant, Children/Teens and Adult and Not precised if NaN.

df.loc[df['Category'].str.contains('hommes?|femmes?', na = False, case = False),'Age Group'] = 'Adult'

df.loc[df['Category'].str.contains('gar[cç]ons?|filles?|enfants?', na = False, case = False),'Age Group'] = 'Children/Teens'

df.loc[df['Category'].str.contains('b[ée]b[ée]', na = False, case = False),'Age Group'] = 'Infant'

age_group_na = {'Age Group':'Not precised'}
df.fillna(value = age_group_na, inplace = True)

In [24]:
# Since the 'Category' columns contain information about 'Gender'
# Create a new column:'Gender' containing 3 values: M and F and Not precised if NaN.
df.loc[df['Category'].str.contains('hommes?|gar[cç]ons?', na = False, case = False),'Gender'] = 'M'

df.loc[df['Category'].str.contains('femmes?|filles?', na = False, case = False),'Gender'] = 'F'

gender_na = {'Gender':'Not precised'}
df.fillna(value = gender_na, inplace = True)

In [25]:
df[df['Category'].str.contains('b[ée]b[ée]', na = False)].sample(5)

,Id Product,Name Product,Brand,Category,Color,Original Price,Buy Online,Current Price,Current Price Excl VAT,Vendor,Discounted Product,Club Discounted Product,Age Group,Gender
Number,,,,,,,,,,,,,,
176,000000000001479490,AIR MAX EXCEE,NIKE,Chaussures / Enfant Loisirs / Bébé / Chaussure...,Noir,NaN,True,59.99,59.99,GSP,False,No,Infant,Not precised
271,000000000001416212,"HOOPS MID 2.0 I,",ADIDAS,Chaussures / Enfant Loisirs / Bébé / Chaussure...,Noir/rouge,NaN,True,39.99,39.99,GSP,False,No,Infant,Not precised
58,000000000001451040,ADVANTAGE I BLANC,ADIDAS,Chaussures / Enfant Loisirs / Bébé / Chaussure...,Blanc/vert,NaN,True,29.99,29.99,GSP,True,Yes,Infant,Not precised
2002,000000000001485856,FRIENDS,ARENA,Sports / Natation - Piscine / Maillot de Bain ...,NaN,NaN,True,14.99,14.99,GSP,False,No,Infant,Not precised
219,000000000001466577,REVOLUTION 5 VLC,NIKE,Chaussures / Enfant Loisirs / Bébé / Chaussure...,Noir,NaN,True,32.99,32.99,GSP,False,No,Infant,Not precised


In [26]:
# Clean the data in the column 'Category':
# remove Homme, femme, garçon, fille, enfant, bébé from the string

df['Category'] = df['Category'].str.replace(r'\s([Hh]ommes?|[Ff]emmes?|[Ff]illes?|[Gg]ar[cç]ons?|[Ee]nfants?|([Bb][ée]){2})\s?','')
df['Category'] = df['Category'].str.replace(r'//','/')

In [36]:
type(df['Category'].unique()[0])

str

In [33]:
df1 = df.copy()

In [35]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2544 entries, 1 to 2544
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id Product               2544 non-null   object 
 1   Name Product             2544 non-null   object 
 2   Brand                    2544 non-null   object 
 3   Category                 2368 non-null   object 
 4   Color                    1983 non-null   object 
 5   Original Price           60 non-null     float64
 6   Buy Online               2544 non-null   bool   
 7   Current Price            2544 non-null   float64
 8   Current Price Excl VAT   2544 non-null   float64
 9   Vendor                   2544 non-null   object 
 10  Discounted Product       2544 non-null   bool   
 11  Club Discounted Product  2544 non-null   object 
 12  Age Group                2544 non-null   object 
 13  Gender                   2544 non-null   object 
dtypes: bool(2), float64(3), 

**Dealing with the column 'Category'**

In [67]:
df1['category_1'] = df1['Category'].map(lambda x: str(x).split('/')[0])

In [69]:
df1['category_1']=df1['category_1'].str.rstrip(' ')

In [70]:
df1['category_1'].unique()

array(['Football', 'Accessoires', 'Sports', 'Chaussures', 'Vêtements',
       'Marques', 'Running', 'nan', 'Fitness - Musculation'], dtype=object)

In [186]:
df1['Category'].str.split('/').str[1]

Number
1                      Clubs 
2                      Clubs 
3         Sacs &amp; Besaces 
4         Sacs &amp; Besaces 
5                      Clubs 
                ...          
2540     Chaussures Crampons 
2541         Accessoires Foot
2542                   Sport 
2543          Sports d'Hiver 
2544                   Sport 
Name: Category, Length: 2544, dtype: object

In [187]:
df1['sub_category'] = df1['Category'].str.split('/').str[1]
df1['sub_category'] = df1['sub_category'].str.strip(' ')

In [189]:
df1['sub_category'].unique()

array(['Clubs', 'Sacs &amp; Besaces', 'Outdoor', 'Loisirs', 'Loisir',
       'Nos Sélections', 'Chaussures', 'Trail', nan, 'Glisse Urbaine',
       'Chaussures Crampons', 'Randonnée', "Sports d'Hiver",
       'Accessoires Multisport', 'Accessoires Foot', "Sports d'Équipe",
       'Sport', 'Tenues', 'Plus de Sports', 'Sélection Sneakers',
       'Sports de Raquette', "Sports d'eau", 'Equipements',
       'Accessoires Textile Adulte', 'Soin - Protection Du Sportif',
       'Alimentation Du Sportif', 'Natation - Piscine', 'Cycle',
       'Nutrition Sportive', 'Accessoires Textile', 'Coupe du monde',
       'Accessoires', 'Nations', 'Vetements', 'Athletisme'], dtype=object)

In [190]:
df1.drop(columns = 'Category', inplace = True)

In [191]:
df1.head()

,Id Product,Name Product,Brand,Color,Original Price,Buy Online,Current Price,Current Price Excl VAT,Vendor,Discounted Product,Club Discounted Product,Age Group,Gender,category_1,sub_category
Number,,,,,,,,,,,,,,,
1,000000000001473525,PSG DOMICILE 2020-2021,NIKE,Bleu,NaN,True,89.99,89.99,GSP,False,No,Not precised,Not precised,Football,Clubs
2,000000000001480797,PSG MAILLOT EXTERIEUR 20,NIKE,Blanc,NaN,True,89.99,89.99,GSP,False,No,Not precised,Not precised,Football,Clubs
3,000000000001147056,PADDED NOIR,EASTPAK,Noir,NaN,True,49.99,49.99,GSP,True,Yes,Not precised,Not precised,Accessoires,Sacs &amp; Besaces
4,000000000001112232,PADDED GRIS,EASTPAK,Gris,NaN,True,49.99,49.99,GSP,True,Yes,Not precised,Not precised,Accessoires,Sacs &amp; Besaces
5,000000000001475281,OM MAILLOT EXTERIEUR 20,PUMA,Bleu ciel,NaN,True,89.99,89.99,GSP,False,No,Not precised,Not precised,Football,Clubs


# Export dataframe from Python into MySQL 

In [205]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from getpass import getpass

In [206]:
hostname = 'localhost'
username = 'root'
password = getpass()

········


In [207]:
db = 'go_sports'

In [208]:
engine = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}/{db}')

In [209]:
#Export the clean DataFrame to MySQL - database 'go_sports'

df1.to_sql('new_product_v1', con=engine, if_exists = 'append')